# Librerías

In [31]:
from selenium import webdriver
import requests
import time
import pandas as pd
import numpy as np

# Funciones

In [58]:
def getDriver(expediente):

    pageURL ='https://appscvsmovil.supercias.gob.ec/portaldeinformacion/consulta_cia_menu.zul?expediente='+expediente+'&tipo=1'
    options = webdriver.ChromeOptions()
    options.add_argument('start-maximized')    
    driver = webdriver.Chrome(options=options,executable_path=r"chromedriver.exe")    
    empresa=''
    
    try:
        driver.get(pageURL)
        site_key = "6LfzwakUAAAAAPmFRhQ3aPRktzKzH-L81F-_Z0bC"

        with open(r"api_key.txt", "r") as f:
          api_key = f.read()

        form = {"method": "userrecaptcha",
                "googlekey": site_key,
                "key": api_key, 
                "pageurl": pageURL, 
                "json": 1}

        response = requests.post('http://2captcha.com/in.php', data=form)
        request_id = response.json()['request']

        url = f"http://2captcha.com/res.php?key={api_key}&action=get&id={request_id}&json=1"

        status = 0
        while not status:
            res = requests.get(url)
    #        print(res)
            if res.json()['status']==0:
                time.sleep(3)
            else:
                requ = res.json()['request']
                js = f'document.getElementById("g-recaptcha-response").innerHTML="{requ}";'
                driver.execute_script(js)
                fjs= f'verifyCallback("{requ}");'
                driver.execute_script(fjs)
                status = 1
        time.sleep(3)

        driver.find_element_by_xpath('//td[@class="z-button-cm"]').click()    
        #driver.save_screenshot('screenshot.png')
        empresa=driver.find_element_by_xpath('//span[@class="z-label"]').text
        arbolAccionistas=driver.find_element_by_xpath('(//a[@class="m_iconos"])[7]')
        driver.execute_script("arguments[0].click()",arbolAccionistas)
        time.sleep(2)
        estado=0
    except:
        estado=1
    return driver,empresa,estado


def getAccionistas(driver,expediente,empresa):
    numeroColumnas=7
    datos=driver.find_elements_by_xpath('//div[contains(@class,"z-treecell-cnt")]')
    datosAccionista=[dato.text for dato in datos]
    Accionista = pd.DataFrame(np.array(datosAccionista).reshape(int(len(datosAccionista)/numeroColumnas),numeroColumnas),
                    columns = ["numero","identificacion","accionista","nacionalidad","tipo_inversion","capital","restriccion"])
    #formato de números a las columnas
    Accionista[Accionista.columns[5]] = Accionista[Accionista.columns[5]].apply(lambda x: x.replace(',','')).astype(float)
    Accionista['expediente']=int(expediente)
    Accionista['empresa']=empresa
    time.sleep(1)    
    driver.close()
    return Accionista  

# Programa Principal

In [60]:
Accionistas=[]
Expedientes=['156811','151630','4164','1195']

for expediente in Expedientes:
    driver,empresa,estado=getDriver(expediente)
    if estado==0:
        Accionista=getAccionistas(driver,expediente,empresa)
        Accionistas.append(Accionista)

AccionistasEmpresa=pd.concat(Accionistas,ignore_index=True)
AccionistasEmpresa.to_csv("Accionista_Compañía.csv")